###### Santiago Tarrío. Alberto Ríos. ISW1 - Inteligencia Artificial

# Trabajo sobre Redes Neuronales

### Implementación de una red neuronal para el reconocimiento de letras manuscritas.

In [1]:
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from os import listdir
from os.path import isfile, join

In [2]:
def img_converter(path,umbral):
    image=Image.open(path)
    image_array=np.asarray(image)
    image_norm=np.asarray([i/255 for i in image_array])
    img_letra = image_norm.tolist()
    img_primera=[]
    for i in img_letra:
        img_primera+=i
    for i in range(len(img_primera)):
        img_primera[i]=1.0 if abs(1-img_primera[i])>=umbral else 0.0
    return img_primera

            
def sigmoide(z):
    s = 1.0/(1.0 + np.exp(-z))
    return s
    
def sigmoide_prima(z):
    s = sigmoide(z)*(1 - sigmoide(z))
    return s

def letra_respuesta(salidas):
    maximo=max(salidas)
    if salidas.index(maximo)==0:
        return "P"
    elif salidas.index(maximo)==1:
        return "Q"
    elif salidas.index(maximo)==2:
        return "R"
    elif salidas.index(maximo)==3:
        return "S"
    elif salidas.index(maximo)==4:
        return "T"
    elif salidas.index(maximo)==5:
        return "U"
    else:
        return "none"

### Conjuntos de entrenamiento y pruebas.
Aquí preparamos los conjuntos que servirán para entrenar a la red y validar su eficiencia, leyendo los archivos directamente del directorio:

In [3]:
path_e="/home/hukgar/Universidad/IA/Trabajo/Casos/CE/"
path_p="/home/hukgar/Universidad/IA/Trabajo/Casos/Prueba/"
archivos_entrenamiento=[path_e+f for f in listdir(path_e) 
                        if isfile(join(path_e, f))]
archivos_entrenamiento=sorted(archivos_entrenamiento, key=lambda x: int(x.split('/')[8].split('.')[0]))
archivos_pruebas=[path_p+f for f in listdir(path_p) 
                        if isfile(join(path_p, f))]
archivos_pruebas=sorted(archivos_pruebas, key=lambda x: int(x.split('/')[8].split('.')[0]))
se=[img_converter(f,0.7) for f in archivos_entrenamiento]

outputs=[[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],
         [0,0,0,0,0,1],[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],
         [0,0,0,0,1,0],[0,0,0,0,0,1],[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],
         [0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1],[0,0,0,0,0,1],[0,0,1,0,0,0],
         [0,1,0,0,0,0],[0,0,0,1,0,0],[1,0,0,0,0,0],[0,0,0,0,1,0],[1,0,0,0,0,0],
         [1,0,0,0,0,0],[0,0,1,0,0,0],[0,1,0,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],
         [0,0,0,1,0,0],[0,0,0,0,0,1],[0,0,0,1,0,0],[0,0,1,0,0,0],[0,0,0,0,1,0],
         [0,1,0,0,0,0]]

Estas son las dos clases principales, que implementan tanto la clase __Neurona__, como la red en sí, que incluye entre sus métodos el _algoritmo de retropropagación_:

In [4]:
class Neuron:
    def __init__(self,bias):
        self.ini = bias*(-1)
        self.error=0.0
        self.a=0.0
        self.bias = bias
        
    def calcula_entrada(self,neuronas_p,pesos_p):
        self.ini=self.bias*(-1)
        for n in range(len(neuronas_p)):
            self.ini+=neuronas_p[n].get_a()*pesos_p[n]
            
    def calcula_salida(self):
        self.a=sigmoide(self.ini)
    
    def calcula_error_ultima_capa(self,Y):
        self.error=self.a*(1-self.a)*(Y-self.a)
        
    def calcula_error(self,neuronas_s,pesos_s):
            suma_p=0.0
            for i in range(len(neuronas_s)):
                suma_p+=pesos_s[i]*neuronas_s[i].get_error()
            self.error=self.a*(1-self.a)*suma_p
    
    def set_output(self,n):
        self.a=n
    def set_input(self,n):
        self.ini=n
    def set_bias(self,n):
        self.bias=n
    def actualiza_bias(self,rate):
        self.bias+=rate*(-1)*self.error
    
    def get_bias(self):
        return self.bias
        
    def get_a(self):
        return self.a
    def get_entrada(self):
        return self.ini
    
    def get_error(self):
        return self.error
        
class NeuralNetwork:
    def __init__(self,neurons,rate,weights,bw_o,bw_s):
        self.weights = weights
        self.rate = rate
        self.neurons=[]
        for i in range(len(neurons)-1):
            self.neurons.append([])
            for j in range(neurons[i]):
                self.neurons[i].append(Neuron(bw_o))
        self.neurons.append([])
        for i in range(neurons[len(neurons)-1]):
            self.neurons[len(neurons)-1].append(Neuron(bw_s))
        self.layers = len(self.neurons) #numero de capas          
    
    def train(self,inX,outY,iterations):
        error=0.0
        for j in range(iterations):
            if j%100==0:
                print("Epoch: {0}".format(j))
            for i in range(len(inX)):
                self.feedforward(inX[i])
                self.backpropagation(outY[i])
            if j==iterations-1:
                print("Fin del entrenamiento.")
            
    def feedforward(self, X):
        for n in range(len(self.neurons[0])):
            self.neurons[0][n].set_output(X[n])
        for i in range(1,self.layers):
            for j in range(len(self.neurons[i])):
                pesos_i=self.weights[i-1][j]
                self.neurons[i][j].calcula_entrada(self.neurons[i-1],pesos_i)
                self.neurons[i][j].calcula_salida()
                
    def backpropagation(self,Y):
        for n in range(len(self.neurons[len(self.neurons)-1])):
            self.neurons[len(self.neurons)-1][n].calcula_error_ultima_capa(Y[n])
            self.neurons[len(self.neurons)-1][n].actualiza_bias(self.rate)
        
        for i in range(self.layers-2,-1,-1):
            for j in range(len(self.neurons[i])):
                pesos_r=[x[j] for x in self.weights[i]]
                self.neurons[i][j].calcula_error(self.neurons[i+1],pesos_r)
                self.neurons[i][j].actualiza_bias(self.rate)
                for k in range(len(self.neurons[i+1])):
                    self.weights[i][k][j]+=self.rate*self.neurons[i][j].get_a()*self.neurons[i+1][k].get_error()
                
    
    def prediccion(self,X):
        for i in range(self.layers):
            for i in self.neurons[i]:
                i.set_input(0.0)
        self.feedforward(X)
        return [self.neurons[2][i].get_a() for i in range(len(self.neurons[2]))]
      
        
    def get_pesos(self):
        return self.weights
    
    def get_neurons(self):
        return self.neurons
    
    def set_rate(self,rate):
        self.rate=rate

In [5]:
neuronas = [900,4,3,6]
pesos = [[[random.random()/10 for i in range(neuronas[0])] for i in range(neuronas[1])],
         [[random.random()/10 for i in range(neuronas[1])] for i in range(neuronas[2])],
         [[random.random()/10 for i in range(neuronas[2])] for i in range(neuronas[3])]]
red = NeuralNetwork(neuronas,0.01,pesos,7,1)

In [7]:
red.get_neurons()[2][1].calcula_salida()
red.rate
red.set_rate(0.1)
red.rate

0.1

In [413]:
print(red.get_neurons()[2][0].get_a())
red.get_neurons()[2][0].get_bias()

0.0


1

In [495]:
(red.get_pesos()[1][2])

[3.4198657194186772,
 -0.60249106672396424,
 1.6434447810274235,
 -2.9133557298758364]

In [ ]:
red.train(se,outputs,2000)

Epoch: 0
Epoch: 100
Epoch: 200
Epoch: 300

In [171]:
objetivo_pruebas=[[0,0,0,1,0,0],[1,0,0,0,0,0],[0,0,0,0,0,1],[0,0,0,1,0,0],
                 [0,1,0,0,0,0],[1,0,0,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],
                 [0,0,1,0,0,0],[0,0,0,0,0,1],[0,0,0,1,0,0],[0,0,1,0,0,0],
                 [1,0,0,0,0,0],[0,1,0,0,0,0]]

In [504]:
print([red.get_neurons()[2][i].get_bias() for i in range(len(red.get_neurons()[2]))])

[1.379696385121429, 4.6080978873700769, 3.4987488245637137, 1.1298761908976549, -0.63945228601741888, 1.6013246440032249]


In [498]:
salida_pruebas=[]
def mostrar_prediccion(red_n,path,umbral):
    entradas=img_converter(path,umbral)
    valores=red_n.prediccion(entradas)
    salida_pruebas.append(valores)
    print("Salida de la red: {0}".format(valores))
    print("Interpretación: {0} \n".format(letra_respuesta(valores)))
    imagen=Image.open(path)
    imagen.show()

In [501]:
salida_pruebas=[]
for i in archivos_pruebas:
    print("Caso {0}: ".format(archivos_pruebas.index(i)+1))
    mostrar_prediccion(red,i,0.7)

Caso 1: 
Salida de la red: [0.042090977609455286, 0.085179775168858815, 0.014194779894684711, 0.83724352132557123, 0.064653428242980385, 0.064202238243590545]
Interpretación: S 

Caso 2: 
Salida de la red: [0.78738697294984084, 0.028041200405275808, 0.13426589163031444, 0.050777938446831372, 0.073880338677010979, 0.039180872602520213]
Interpretación: P 

Caso 3: 
Salida de la red: [0.059383783433549477, 0.11687684315628523, 0.0016555277256781219, 0.055035608813520015, 0.13464250555624849, 0.87546151632624014]
Interpretación: U 

Caso 4: 
Salida de la red: [0.079730158843824686, 0.48483949448478175, 0.016353894808004827, 0.258769518064495, 0.0025122037296252666, 0.10941711885036644]
Interpretación: Q 

Caso 5: 
Salida de la red: [0.010810903328317318, 0.865255526352122, 0.10005468364239178, 0.037870311980650891, 0.0012085993075358296, 0.057753010484717641]
Interpretación: Q 

Caso 6: 
Salida de la red: [0.47484930358572275, 0.050369445489268444, 0.45740272923921704, 0.021321945487825775

In [461]:
def rendimiento(salidas,objetivos):
    total=len(objetivos)
    acertadas=0
    for i in range(len(objetivos)):
        ms=np.argmax(salidas[i])
        mo=np.argmax(objetivos[i])
        if ms==mo:
            acertadas+=1
    return "Rendimiento: {0} %".format((acertadas/total)*100)

##### 

In [502]:
rendimiento(salida_pruebas,objetivo_pruebas) 

'Rendimiento: 92.85714285714286 %'